# Graphs
Implementing a graph in Python, using an adjacency list representation, implemented with dictionaries.

Our graph uses a dictionary which keys are all the vertices and which values are dictionaries themselves. Those dictionaries have an entry for each connected vertex, which value is the connecting edge.

